In [32]:
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras import layers
import tensorflow as tf
import time

In [33]:
tf.__version__

'2.11.0'

In [34]:
from tensorflow.keras.utils import to_categorical as labelEncoding 

T = 15 # terminus_length

X1 = np.load('bpf-740.npy')


X1 = X1[:376,:T,:]


Y  = [1 for _ in range(376)]

Y = labelEncoding(Y, dtype=int)

print(X1.shape)
print(Y.shape)

(376, 15, 20)
(376, 2)


In [35]:
print(X1.shape)
print(X1[0].shape)
print(X1[0])

BUFFER_SIZE = 1000
BATCH_SIZE = 32

train_dataset = tf.data.Dataset.from_tensor_slices(X1).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

(376, 15, 20)
(15, 20)
[[0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]]


In [36]:
# Deep Neural Networks:
import tensorflow as tf; print('We\'re using TF-{}.'.format(tf.__version__))
# import keras; print('We\'re using Keras-{}.'.format(keras.__version__))
from tensorflow.keras.layers import (Input, Dense, Dropout, Flatten, BatchNormalization,
                                     Conv1D, Conv2D, MaxPooling1D, MaxPooling2D,
                                     LSTM, GRU, Embedding, Bidirectional, Concatenate)
from tensorflow.keras.regularizers import (l1, l2, l1_l2)
from tensorflow.keras.optimizers import (RMSprop, Adam, SGD)
from tensorflow.keras.models import (Sequential, Model)

# Core:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import interp
import matplotlib.patches as patches

# Performance:
from sklearn.metrics import (confusion_matrix, classification_report, matthews_corrcoef, precision_score, roc_curve, auc)
from sklearn.model_selection import (StratifiedKFold, KFold, train_test_split)

#Utilities:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import to_categorical as labelEncoding   # Usages: Y = labelEncoding(Y, dtype=int)
from tensorflow.keras.utils import plot_model   

We're using TF-2.11.0.


In [37]:
def build_discriminator():
    ### Head-1:
    input1 = Input(shape=X1[0].shape)

    x = Conv1D(filters=10, kernel_size=4, padding='same', activation='relu', kernel_regularizer=l2(l=0.01))(input1)
    x = BatchNormalization()(x)
    x = Dropout(rate=0.80)(x)

    x = Conv1D(filters=8, kernel_size=3, padding='same', activation='relu', kernel_regularizer=l2(l=0.01))(x)
    x = BatchNormalization()(x)
    x = Dropout(rate=0.70)(x)

    head1 = Flatten()(x)

    # merge
    merge = Concatenate()([head1])

    output = Dense(units=8, activation='relu', kernel_regularizer=l2(l=0.01))(merge)
    output = BatchNormalization()(output)
    output = Dropout(rate=0.70)(output)

    output = Dense(units=2, activation='softmax')(output)

    return Model(inputs=[input1], outputs=[output])

In [38]:
from Bio.Seq import Seq
from Bio.SeqUtils import ProtParam
from Bio.Align import substitution_matrices
from Bio.SubsMat import MatrixInfo


# Function to convert binary profile feature to amino acid sequence
def bpf_to_sequence(binary_profile_feature):
    amino_acids = "ARNDCQEGHILKMFPSTWYVBZX"
    sequence = ""
    for row in binary_profile_feature:
        index = row.argmax()
        sequence += amino_acids[index]
    return sequence

def seq_to_blosum(sequence):
    blosum62 = MatrixInfo.blosum62
    sequence = sequence.upper()
    length = len(sequence)
    blosum_matrix = []
    for aa1 in sequence:
        row = []
        for aa2 in sequence:
            if (aa1, aa2) in blosum62:
                row.append(blosum62[(aa1, aa2)])
            else:
                row.append(blosum62[(aa2, aa1)])
        blosum_matrix.append(row)
    return blosum_matrix

In [39]:
def bpf_to_sequence(binary_profile_feature):
    amino_acids = "ARNDCQEGHILKMFPSTWYVBZX"
    sequence = ""
    for row in binary_profile_feature:
        i = 0
        for item in row:
            i += 1
            if item.numpy() == 1:
                sequence += amino_acids[i]
    return sequence

def seq_to_blosum(sequence):
    blosum62 = MatrixInfo.blosum62
    sequence = sequence.upper()
    length = len(sequence)
    blosum_matrix = []
    for aa1 in sequence:
        row = []
        for aa2 in sequence:
            if (aa1, aa2) in blosum62:
                row.append(blosum62[(aa1, aa2)])
            else:
                row.append(blosum62[(aa2, aa1)])
        blosum_matrix.append(row[:20])
    return blosum_matrix

def bpf_to_blosum_layer(seqs_bpf):
    seqs_blosum = []
    for bpf in seqs_bpf:
        str_seq = bpf_to_sequence(bpf)
        seqs_blosum.append(seq_to_blosum(str_seq))

    print(seqs_blosum)
    return seqs_blosum

def build_generator():
    model = tf.keras.Sequential()
    model.add(layers.Dense(256, use_bias=False, input_shape=(100,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Dense(512))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Dense(25 * 20, activation='sigmoid'))
    model.add(layers.Reshape((25, 20)))
    return model
    

    # model = Sequential([
    #     layers.Input(shape=(latent_dim,)),
    #     layers.Dense(128, activation='relu'),
    #     layers.Dense(256, activation='relu'),
    #     layers.Dense(output_shape[0] * output_shape[1], activation='sigmoid'),
    #     layers.Reshape(output_shape),
    #     layers.Lambda(lambda x: tf.one_hot(tf.argmax(x, axis=-1), depth=output_shape[1])),
    #     layers.Lambda(lambda x: tf.cast(x, dtype=tf.int32)),
    # ])

    # return model

In [40]:
# Define the dimensions
latent_dim = 100  # Dimensionality of the latent space
batch_size = 1
output_shape = (25, 20)  # Desired output shape of the matrix

generator = build_generator()

noise = tf.random.normal([batch_size, latent_dim])

generated_matrix = generator(noise, training=False)

generated_matrix

<tf.Tensor: shape=(1, 25, 20), dtype=float32, numpy=
array([[[0.4177503 , 0.5410225 , 0.5191743 , 0.4092115 , 0.5264865 ,
         0.5068796 , 0.4981922 , 0.3752302 , 0.39406362, 0.375696  ,
         0.65833527, 0.48286122, 0.5388575 , 0.47444177, 0.5417432 ,
         0.49155676, 0.6065807 , 0.6115332 , 0.5164627 , 0.47408876],
        [0.2792052 , 0.56848186, 0.5060081 , 0.68393797, 0.46552604,
         0.57585406, 0.6001385 , 0.41196012, 0.4627222 , 0.41055036,
         0.50699484, 0.37217724, 0.43834338, 0.53974426, 0.5383152 ,
         0.7316912 , 0.39973783, 0.4404458 , 0.49866048, 0.38269988],
        [0.42645922, 0.4642695 , 0.53303045, 0.4519937 , 0.38793388,
         0.6316519 , 0.64925116, 0.61246985, 0.5330645 , 0.54738283,
         0.42506808, 0.3343812 , 0.37869853, 0.5177621 , 0.43524584,
         0.40246683, 0.63195807, 0.5087882 , 0.5352669 , 0.4144797 ],
        [0.43680716, 0.51767695, 0.579431  , 0.49107927, 0.4525455 ,
         0.48567742, 0.67157775, 0.5889723 , 0.

In [41]:
discriminator = build_discriminator()
discriminator.load_weights('./acp_mhcnn_weights.h5')

In [42]:
prediction = discriminator.predict([X1[:5,:,:]])
prediction

1/1 [==============================] - 0s 70ms/step


array([[0.3078078 , 0.6921922 ],
       [0.17504196, 0.824958  ],
       [0.31832415, 0.6816759 ],
       [0.18360282, 0.81639725],
       [0.19149974, 0.80850023]], dtype=float32)

In [43]:
noise = tf.random.normal([1000, 100])
fake_seqs = generator(noise, training=False)

prediction = discriminator.predict([fake_seqs[:,:15,:]])
print(prediction)

over50 = 0
over25 = 0
over40 = 0
total = 0
for key, output in enumerate(prediction):
    if output[1] > 0.25:
        over25 += 1
    if output[1] > 0.4:
        over40 += 1
    if output[1] > 0.5:
        over50 += 1
    total += 1

print(over25 / total)
print(over40 / total)
print(over50 / total)

32/32 [==============================] - 0s 2ms/step
[[9.9999666e-01 3.3694214e-06]
 [9.9999964e-01 3.4618466e-07]
 [9.9999976e-01 2.8348268e-07]
 ...
 [9.9999976e-01 2.2222730e-07]
 [9.9999964e-01 3.9724043e-07]
 [9.9999952e-01 4.9980542e-07]]
0.0


In [44]:
def discriminator_loss(real_output, fake_output):
    real_loss = tf.keras.losses.BinaryCrossentropy()(tf.ones_like(real_output), real_output)
    fake_loss = tf.keras.losses.BinaryCrossentropy()(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

In [45]:
def generator_loss(fake_output):
    return tf.keras.losses.BinaryCrossentropy()(tf.ones_like(fake_output), fake_output)

# def generator_loss(disc_output):
#     batch_size = tf.shape(disc_output)[0]
#     num_classes = tf.shape(disc_output)[1]
#     desired_output = tf.concat([tf.zeros((batch_size, 1)), tf.ones((batch_size, num_classes - 1))], axis=1)
    
#     # Define binary cross entropy loss
#     bce = tf.keras.losses.BinaryCrossentropy(from_logits=False)
    
#     # Calculate loss
#     loss = bce(desired_output, disc_output)
    
#     return loss

In [46]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [47]:
noise = tf.random.normal([5, 100])
fake_seqs = generator(noise, training=False)

fake_prediction = discriminator.predict([fake_seqs[:,:15,:]])
real_prediction = discriminator.predict([X1[:5]])
print(fake_prediction)
print(real_prediction)

# disc_loss = discriminator_loss(Y[:5], real_prediction)
gen_loss = generator_loss(fake_prediction)
print(gen_loss)
disc_loss = discriminator_loss(real_prediction, fake_prediction)
print(disc_loss)

1/1 [==============================] - 0s 17ms/step
[[9.9999940e-01 5.4496769e-07]
 [9.9999738e-01 2.6562680e-06]
 [9.9999702e-01 3.0341221e-06]
 [9.9999964e-01 4.0852419e-07]
 [9.9999952e-01 5.0236633e-07]]
[[0.3078078  0.6921922 ]
 [0.17504196 0.824958  ]
 [0.31832415 0.6816759 ]
 [0.18360282 0.81639725]
 [0.19149974 0.80850023]]
tf.Tensor(6.8543024, shape=(), dtype=float32)
tf.Tensor(7.7417684, shape=(), dtype=float32)


In [48]:
import os

checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

In [50]:
BUFFER_SIZE = 1000
BATCH_SIZE = 32

EPOCHS = 500
noise_dim = 100

In [53]:
@tf.function
def train_step(peptides):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_bpf = generator(noise, training=True)

      real_output = discriminator(peptides, training=True)
      fake_output = discriminator(generated_bpf[:,:15,:], training=True)

      gen_loss = generator_loss(fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    print(gen_loss)
    print(disc_loss)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

def train(epochs):
  for epoch in range(epochs):
    dataset = tf.data.Dataset.from_tensor_slices(X1).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

    start = time.time()

    for seq_batch in dataset:
      train_step(seq_batch)

    # Save the model every 15 epochs
    if (epoch + 1) % 15 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)

    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))


In [54]:
train(EPOCHS)

Tensor("binary_crossentropy/weighted_loss/value:0", shape=(), dtype=float32)
Tensor("add:0", shape=(), dtype=float32)
Tensor("binary_crossentropy/weighted_loss/value:0", shape=(), dtype=float32)
Tensor("add:0", shape=(), dtype=float32)
Time for epoch 1 is 5.514289855957031 sec
Time for epoch 2 is 0.855933427810669 sec
Time for epoch 3 is 1.1011767387390137 sec
Time for epoch 4 is 1.3729007244110107 sec
Time for epoch 5 is 1.252692461013794 sec
Time for epoch 6 is 1.2778620719909668 sec
Time for epoch 7 is 1.208526611328125 sec
Time for epoch 8 is 1.6376838684082031 sec
Time for epoch 9 is 1.9902801513671875 sec
Time for epoch 10 is 1.2782330513000488 sec
Time for epoch 11 is 1.476546287536621 sec
Time for epoch 12 is 1.4736523628234863 sec
Time for epoch 13 is 1.51523756980896 sec
Time for epoch 14 is 1.186385154724121 sec
Time for epoch 15 is 1.3547885417938232 sec
Time for epoch 16 is 1.5234668254852295 sec
Time for epoch 17 is 1.2951395511627197 sec
Time for epoch 18 is 1.3324706554

In [61]:
noise = tf.random.normal([1000, 100])
fake_seqs = generator(noise, training=False)

prediction = discriminator.predict([fake_seqs[:,:15,:]])
print(prediction)

over50 = 0
over25 = 0
over40 = 0
total = 0
for key, output in enumerate(prediction):
    if output[1] > 0.25:
        over25 += 1
    if output[1] > 0.4:
        over40 += 1
    if output[1] > 0.5:
        over50 += 1
    total += 1

print(over25 / total)
print(over40 / total)
print(over50 / total)

32/32 [==============================] - 0s 4ms/step
[[0.5454503  0.45454973]
 [0.47031873 0.52968127]
 [0.54570436 0.45429558]
 ...
 [0.49318314 0.50681686]
 [0.5340632  0.46593684]
 [0.49426886 0.50573117]]
1.0
0.993
0.139
